# ACO - ANT COLONY OPTIMIZATION 
# Leonardo Monsalvo

In [8]:
import string, math
import numpy as np
from copy import deepcopy
import random

# Matriz de adyacencia, distancias
# Matriz de Heuristica local [nij]

In [9]:
'''
Returns an adjacency matrix from a tsp dataset.
'''
def distancesFromCoords():
    f = open('kroA100.tsp')
    #Reemplazar los saltos por "" y elimina los espacios en blancos por comas en cierta posición
    data = [line.replace("\n","").split(" ")[1:] for line in f.readlines()[6:106]]#Se leen las 100 ciudades
    #Al hacer map recibe como parametros un lambda (función) y data (datos que recibira y hará la función lambda)
    coords =  list(map(lambda x: [float(x[0]),float(x[1])], data))
    distances = []
    for i in range(len(coords)):
        row = []
        for j in range(len(coords)):
            row.append(math.sqrt((coords[i][0]-coords[j][0])**2 + (coords[i][1]-coords[j][1])**2))
        distances.append(row)
    return (distances)

import warnings
warnings.simplefilter("ignore")

def locH():#Heuristica local matriz
    mat_dist = distancesFromCoords()
    h = mat_dist.copy()
    for i in range (len(mat_dist)):
        for j in range (len(mat_dist)):
            if (mat_dist[i][j] != 0): #Si los valores de las distancias 
                
                try:
                    h[i][j]= (1/mat_dist[i][j])
                except ZeroDivisionError:
                    print("")
    
    return (np.array(h))

# Ruta que las hormigas toman 
## Se toman rutas aleatorias con base a la nuestra matriz de adyacencia 
## Se asigna la cantidad de hormigas exploradoras para la creación de la matriz de feromonas 

In [10]:
def Routes():#Movimiento y caminos de la hormiga
    
    mat_dist = distancesFromCoords()
    nciudad = len(mat_dist)#Cantidad de ciudades
    pos = np.random.randint(0,len(mat_dist)-1)
    aux = []
    for i in range (nciudad):
        if (i!=pos): #Si la posición actual es diferente a la ciudad, se añade
            aux.append(i)
    ruta = [pos]

    while (len(aux)>0):
        aux1 = random.choice(aux)
        aux.remove(aux1)
        ruta.append(aux1)
    ruta.append(pos)

    z = Objetive(ruta, mat_dist)
    return (ruta,z)

def firstFero():#Feromonas del primer recorrido
    n = 1000
    mat_h = locH()
    fermat = np.zeros(np.shape(mat_h))#Matriz de igual tamaño con ceros 
    while(n>0):

        soluciones,valor = Routes()
        try:
            matrizfer = 1/valor #
        except ZeroDivisionError:
            print("")

        for i in range(len(soluciones)-1):
            start = soluciones[i]
            end = soluciones[i+1]
            fermat[start][end]+=matrizfer
        n-=1
    return (fermat)

# Sumar la distancias de la matriz de adyacencia, de inicio a fin

In [11]:
def Objetive(x,mat_dist):

    suma = 0
    for i in range(len(x)-1):
        start = x[i] #ciudad de inicio
        end = x[i+1] #ciudad final
        suma+= mat_dist[start][end]
    return (suma)

def camino(mat_h):
    mat_dist = distancesFromCoords()
    mat_fer = firstFero()
    pos = np.random.randint(0,len(mat_dist)-1)
    alfa = 3
    beta = 6
    aux = 0
    posicion = [pos]
    while (len(posicion)<len(mat_fer)):
        aleatorio = random.random()
        probabilidades = Probabilidad(mat_h,mat_fer,alfa,beta,posicion)
        posicion.append(posAcumu(probabilidades[:,posicion[aux]],aleatorio,posicion))
        aux += 1
    posicion.append(pos)
    return (posicion)

#  Pij(t) = (Tij^α x nij^β)/(Σ Tij^α x nij^β) 
## Probabilidad para ir de una ciudad a otra
## Se modifica o actualiza la matriz de feromonas

In [12]:
def Probabilidad(mat_h, mat_fer,alfa,beta,position):

    matriz = (mat_fer**alfa)*(mat_h**beta)
    for i in position:
        matriz[i,:] = 0

    suma1 = matriz.sum(axis=0)
    probabilidad = matriz/suma1
    return (probabilidad)

def plusfero(mat_fer, solucion, mat_dist):

    aux = deepcopy(mat_fer)
    z = Objetive(solucion, mat_dist)
    try:
        matrizfer = 1/z
    except ZeroDivisionError:
        print("")

    for i in range(len(solucion)-1):
        start = solucion[i]
        end = solucion[i+1]
        matrizfer+=aux[start][end]
        
    return np.array(aux)

# Evaporación de las feromnonas con un valor p = 0.1 (índice de evaporación)

In [13]:
def posAcumu(probabilidad,aleatorio,position):

    aux = 0
    for i in range (len(probabilidad)):
        aux += probabilidad[i]
        if (probabilidad[i] in position):
            continue
        elif (aux>aleatorio):
            return i
    return 0

def evaporacion(mat_fer):
    p = 0.1 #Indice de evaporización
    for i in range(len(mat_fer)):
        for j in range(len(mat_fer[0])):
            #Formula para la evaporación
            mat_fer[i][j] = (1-p)*mat_fer[i][j]
    return (mat_fer)

# Llamado de las fuciones

In [14]:
if __name__ == "__main__":
    
    mat_dist = distancesFromCoords()
    mat_h = locH()
    mat_fer = firstFero()
    
    while(True):
        iteraciones = input("Iteraciones: ")
        try:
            iteraciones = int(iteraciones)
            if(iteraciones<0):
                print("\nNumero invalido!\n")
            else:
                break
        except ValueError:
            print("\nDigite un numero!\n")
    
    for i in range (iteraciones):
        
        solucion = camino(mat_h)
        mafe = plusfero(mat_fer,solucion,mat_dist)
        mafe = evaporacion(mat_fer)
        
    print("Ruta total: ")
    print(solucion)
    print("\nValor de z: ", Objetive(solucion,mat_dist))

Iteraciones: 1000
Ruta total: 
[85, 26, 11, 19, 56, 6, 8, 86, 50, 24, 80, 67, 84, 72, 49, 43, 1, 53, 39, 63, 68, 60, 57, 27, 92, 0, 48, 5, 62, 7, 91, 74, 18, 52, 93, 21, 87, 15, 69, 65, 64, 3, 96, 55, 79, 30, 88, 41, 66, 97, 90, 22, 44, 31, 71, 20, 73, 58, 16, 14, 10, 9, 83, 23, 37, 35, 98, 78, 17, 89, 46, 61, 76, 59, 34, 82, 54, 33, 28, 45, 2, 42, 95, 77, 51, 29, 47, 99, 70, 40, 13, 38, 4, 36, 32, 12, 75, 94, 81, 0, 85]

Valor de z:  30426.325329964668
